## Dynamic programming

Dynamic programming can be thought of as an optimization on top of a recursive problem, where we cache intermediate results in a 'dynamic' array. This can be done either bottom-up or top-down (aka memoization). The important thing to understand is that bottom-up and top-down are equivalent, they're just different styles of arriving at the same solution, where bottom-up is arguably more elegant.

## Problem: the Fibonaccy sequence

The Fibonacci sequence is:  
fib(0) = 0  
fib(1) = 1  
fib(n>1) = fib(n-1) - fib(n-2)  

In [1]:
def fib_slow(n):

    # this recursive algorithm scales like 2^n - which is terrible!
    if n==0 or n==1:
        return n
    else: 
        return fib_slow(n-2) + fib_slow(n-1)

In [2]:
%%time

# note how slow this is
fib_slow(30)

CPU times: user 297 ms, sys: 2.67 ms, total: 300 ms
Wall time: 299 ms


832040

In [3]:
def fib_bu(n):
    # bottom-up approach. solve the smaller numbers first, and go higher incrementally
    if n==0 or n==1:
        return n
    a,b = 0,1
    i = 2
    while i<n:
        a,b = b,a+b
        i += 1
    return a+b

In [4]:
%%time

# note how much faster this is
fib_bu(30)

CPU times: user 23 µs, sys: 1e+03 ns, total: 24 µs
Wall time: 31 µs


832040

In [5]:
def fib_memo(n):
    # in the memoization approach, we use a cache to store values we already computed previously
    cache = {}
    def find_fib(n, cache):
        if n==0 or n==1:
            return n
        elif n in cache.keys():
            return cache[n]
        else:
            res = find_fib(n-2,cache) + find_fib(n-1,cache)
            cache[n] = res
        return res
    return find_fib(n, cache)

In [6]:
%%time

# similarly fast as bottom-up approach
fib_memo(30)

CPU times: user 73 µs, sys: 1e+03 ns, total: 74 µs
Wall time: 79.2 µs


832040

## Problem: coin chance

Given coins c, what's the minimum number of coins needed to make amount 'target'?

First, note the mathematical property of the solution, aka the induction rule:

F(x) = min( 1 + F(x-c) ) 

where the min is taken over all c<=x.

Once we have the induction rule, the rest of the code is really straightforwards:

In [16]:
import numpy as np
def coin_change(target,coins):

    dp = [0] + [np.infty]*target
    for x in range(1,target+1):
        candidates = [1+dp[x-c] for c in coins if x-c>=0]
        if len(candidates)>0:
            dp[x] = min(candidates)
        
        else:
            # we can't make x with coins at all!
            dp[x] = -1
        print(dp)  # for illustration purposes
            
    return dp[-1]

In [21]:
coin_change(13, [1,3,7])

[0, 1, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, inf, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, inf, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, inf, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, inf, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, 3, inf, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, 3, 2, inf, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, inf, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 4, inf]
[0, 1, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 4, 3]


3

## Problem: length of the longest increasing sub-sequence

Example: [1,3,2,5,6,4] --> 1,2,5,6 --> 4

Again let's start with the induction rule. Suppose we know dp[j], the lenght of LIS at position j, then what's the result dp[i] for i>j?

Answer: dp[i] = max(dp[i], 1+dp[j]) if nums[i]>nums[j] 

And there we have it. Code:

In [26]:
def lis(nums):
    dp = [1]*len(nums)
    for i in range(1,len(nums)):
        for j in range(0,i):
            if nums[i]>nums[j]:
                dp[i] = max(dp[i],1+dp[j])
        print(dp)
    return max(dp)

In [27]:
lis([1,3,2,5,6,4])

[1, 2, 1, 1, 1, 1]
[1, 2, 2, 1, 1, 1]
[1, 2, 2, 3, 1, 1]
[1, 2, 2, 3, 4, 1]
[1, 2, 2, 3, 4, 3]


4

## Problem: length of the longest common sub-sequence

Given 2 strings a,b return the length of the longest common sub-sequence.

Example: 'abcde','ace' --> 3

Induction rule:
Let dp[i][j] denote the solution for a[i:],b[j:]. Then the following induction holds:

dp[i][j] = 1+dp[i+1][j+1] if a[i]==b[j] else max(dp[i+1][j],dp[i][j+1])

In [30]:
def lcss(a,b):
    dp = [[0]*(1+len(b)) for _ in range(1+len(a))]
    for i in range(len(a))[::-1]:
        for j in range(len(b))[::-1]:
            if a[i]==b[j]:
                dp[i][j] = 1+dp[i+1][j+1]
            else:
                dp[i][j] = max(dp[i+1][j],dp[i][j+1])
    return dp[0][0]

In [33]:
lcss('abcde','ace')

3

## Problem: length of the longest common substring

This is almost the same problem as the longest common sub-sequence with the difference that we're looking for a common contiguous string.

Example: 
'abcde','ace' --> 1 (!)
'abcde','bcd' --> 3

The induction rule here is almost the same as above, with the difference that we don't use the else statement:

Let dp[i][j] be the solution for a[i],b[j]. Then: 

dp[i][j] = 1+dp[i+1][j+1] if a[i]==b[j] (no else statement)

In [36]:
def lcssg(a,b):
    dp = [[0]*(1+len(b)) for _ in range(1+len(a))]
    for i in range(len(a))[::-1]:
        for j in range(len(b))[::-1]:
            if a[i]==b[j]:
                dp[i][j]=1+dp[i+1][j+1]
    return max( [max(row) for row in dp] )

In [37]:
lcssg('abcde','bcd')

3

## Problem: number of combinations to make x

Given nums and x, return the number of combinations of nums that sum to x.

Example: \
9,[3,5] --> 1 \
9,[5] --> 0 \
9,[1,3] --> 19 \
etc

As always, let's first spell out the induction rule:

Lets denote dp[i] the solution for target=i. Then:

**dp[i] = Sum( dp[i-num] for num in nums if (i-num)>=0 )**

In words: in order to arrive at i, consider all (i-num) and sum them.

In [43]:
def num_combis(x,nums):
    dp = [1] + [0]*x
    for i in range(x+1):
        for num in nums:
            if x-num>=0:
                dp[i]+=dp[i-num]
        print(dp)
    return dp[-1]

In [51]:
num_combis(9,[3,2])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 2, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 2, 2, 0, 0, 0]
[1, 0, 1, 1, 1, 2, 2, 3, 0, 0]
[1, 0, 1, 1, 1, 2, 2, 3, 4, 0]
[1, 0, 1, 1, 1, 2, 2, 3, 4, 5]


5